In [2]:
import pandas as pd
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from geojson import Point, Feature, FeatureCollection, dump
import requests
import json
import math
import numpy
import os
import sys
import datetime
import numpy as np

In [3]:
def get_data(url):
    api = requests.get(url)
    print('Complete Fetch', url, " ", api.status_code)
    data = api.text
    js = json.loads(data)
    return pd.DataFrame.from_dict(js)

In [4]:
last_fetch_time = "'2023-01-01T12:00:00.000'"

In [5]:

FSR = get_data("https://data.cityofnewyork.us/resource/mu46-p9is.json?$where=createddate>" + last_fetch_time + "&$limit=1000000")
FI = get_data("https://data.cityofnewyork.us/resource/4pt5-3vv4.json?$where=createddate>" + last_fetch_time + "&$limit=1000000")
FWO = get_data("https://data.cityofnewyork.us/resource/bdjm-n7q4.json?$where=createddate>"+ last_fetch_time + "&$limit=1000000")
FRA = get_data("https://data.cityofnewyork.us/resource/259a-b6s7.json?$where=createddate>" + last_fetch_time + "&$limit=1000000")
# shape = gpd.read_file("visualize/viz_data/nycb2020.shp") # shapefile for census tracts
# https://www.nyc.gov/site/planning/data-maps/open-data/census-download-metadata.page this contains census blocks, used for joining the demographics
len(FSR), len(FI), len(FWO), len(FRA)

Complete Fetch https://data.cityofnewyork.us/resource/mu46-p9is.json?$where=createddate>'2023-01-01T12:00:00.000'&$limit=1000000   200
Complete Fetch https://data.cityofnewyork.us/resource/4pt5-3vv4.json?$where=createddate>'2023-01-01T12:00:00.000'&$limit=1000000   200
Complete Fetch https://data.cityofnewyork.us/resource/bdjm-n7q4.json?$where=createddate>'2023-01-01T12:00:00.000'&$limit=1000000   200
Complete Fetch https://data.cityofnewyork.us/resource/259a-b6s7.json?$where=createddate>'2023-01-01T12:00:00.000'&$limit=1000000   200


(5542, 20526, 7156, 14089)

In [6]:
# FSR.head()["createddate"]
FSR.columns

Index(['objectid', 'complaintnumber', 'srcategory', 'srtype', 'srpriority',
       'srsource', 'srstatus', 'srresolution', 'boroughcode', 'communityboard',
       'buildingnumber', 'streetname', 'crossstreet1', 'crossstreet2',
       'locationdetails', 'zipcode', 'citycouncil', 'statesenate',
       'stateassembly', 'congressional', 'sanitationzone', 'physicalid',
       'geometry', 'globalid', 'initiateddate', 'createddate', 'updateddate',
       'descriptor1', 'complainttype', 'srcallertype', 'latitude', 'longitude',
       'census_tract', 'bin', 'bbl', 'nta', 'location', 'complaintdetails',
       'notestocustomer', 'servicerequestparentglobalid', 'closeddate',
       'callerstate', 'callerzipcode', 'callercity', 'parkname', 'parkzone'],
      dtype='object')

In [ ]:
# shape.GEOID = shape.GEOID.astype(str).str[0:12]
# shape = shape.to_crs({'init': 'epsg:4326'}) ## convert to lat/long

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
# FSR Global ID corresponds to FI Service Request Global ID
len(set(FI.servicerequestglobalid) - set(FSR.globalid)) 

In [ ]:
# FI GLOBAL ID corresponds to FWO Inspection Global ID
len(set(FWO.inspectionglobalid) - set(FI.globalid))

In [ ]:
# FRA IncidentGlobalID corresponds to FI Global ID, roughly...
len(set(FRA.inspectionglobalid) - set(FI.globalid))

In [8]:
FSR_columns = ['srcategory', 'srtype', 'srpriority',
       'srsource', 'srstatus', 'srresolution', 'boroughcode',
       'servicerequestparentglobalid', 'globalid',
       'initiateddate', 'closeddate', 'createddate', 'updateddate',
       'descriptor1', 'complainttype', 'callerzipcode', 'srcallertype',
       'latitude', 'longitude', 'nta', 'communityboard', 'zipcode', 'census_tract']

FI_columns = ['inspectiontype', 'inspectionstatus',
       'inspectiontpcondition', 'inspectiontpstructure', 'treepointdbh',
       'globalid', 'servicerequestglobalid', 'inspectiondate', 'closeddate', 'createddate',
       'updateddate', 'parentinspectionglobalid', 'reinspectiondate', 'location']

FWO_columns = ['wotype', 'wostatus', 'wopriority', 'boroughcode',
       'inspectionglobalid', 'globalid', 'closeddate',
       'canceldate', 'cancelreason', 'createddate', 'updateddate', 'woentity',
       'woproject', 'wocategory', 'projstartdate', 'recommendedspecies', 'location']

FRA_columns = ['objectid', 'radefect', 'radefectlocation', 'failure', 'impacttarget',
       'consequence', 'riskrating', 'inspectionglobalid', 'globalid',
       'createddate', 'failureimpact', 'workorderglobalid']

# Deal with empty return

In [9]:
FI.columns

Index(['objectid', 'inspectiontype', 'inspectionstatus',
       'inspectiontpcondition', 'inspectiontpstructure', 'treepointdbh',
       'treepointglobalid', 'plantingspaceglobalid', 'servicerequestglobalid',
       'geometry', 'globalid', 'inspectiondate', 'closeddate', 'createddate',
       'location', 'updateddate', 'swtreeconditionrating', 'swlocationrating',
       'swclearancerating', 'swdamagerating', 'swdamagedflagstonesrating',
       'swratingtotal', 'swtotaltrees', 'parentinspectionglobalid',
       'reinspectiondate'],
      dtype='object')

In [10]:
#If there's no new service requests, stop the processing
if (len(FSR) == 0):
    sys.exit("Exiting the job as there's no new service requests")

In [11]:
if (len(FI) == 0):
    FI = pd.DataFrame(columns=FI_columns)

In [12]:
if (len(FWO) == 0):
    FWO = pd.DataFrame(columns=FWO_columns)

In [13]:
if (len(FRA) == 0):
    FRA = pd.DataFrame(columns=FRA_columns)

# Only keep subset of data

In [14]:
FSR.columns

Index(['objectid', 'complaintnumber', 'srcategory', 'srtype', 'srpriority',
       'srsource', 'srstatus', 'srresolution', 'boroughcode', 'communityboard',
       'buildingnumber', 'streetname', 'crossstreet1', 'crossstreet2',
       'locationdetails', 'zipcode', 'citycouncil', 'statesenate',
       'stateassembly', 'congressional', 'sanitationzone', 'physicalid',
       'geometry', 'globalid', 'initiateddate', 'createddate', 'updateddate',
       'descriptor1', 'complainttype', 'srcallertype', 'latitude', 'longitude',
       'census_tract', 'bin', 'bbl', 'nta', 'location', 'complaintdetails',
       'notestocustomer', 'servicerequestparentglobalid', 'closeddate',
       'callerstate', 'callerzipcode', 'callercity', 'parkname', 'parkzone'],
      dtype='object')

In [15]:
# insert columns if not exits
FSR = FSR.reindex(columns=FSR_columns)
FI = FI.reindex(columns=FI_columns)
FWO = FI.reindex(columns=FWO_columns).fillna('N/A')
FRA = FRA.reindex(columns=FRA_columns).fillna('N/A')

In [16]:
# select subset due to memory limit
FSR = FSR[FSR_columns]
FI = FI[FI_columns]
FWO = FWO[FWO_columns]
FRA = FRA[FRA_columns]

# transform SR df into geodf

In [17]:
FSR.columns
len(FSR.columns)

23

In [18]:
## incident global ID (deal with parent requests)
FSR.loc[:, "GLOBAL_ID"] = FSR.apply(lambda x:  x.globalid if x.servicerequestparentglobalid!=x.servicerequestparentglobalid else x.servicerequestparentglobalid, axis = 1)

In [19]:
FSR['longitude'] = FSR['longitude'].astype(float)
FSR['latitude'] = FSR['latitude'].astype(float)

In [20]:
#requests that have no geo-info
FSR.longitude.isna().sum(), FSR.latitude.isna().sum() 

(466, 466)

In [21]:
# find census tract
geometry = [None if math.isnan(xy[0]) or math.isnan(xy[1]) else Point(xy) for xy in zip(FSR.longitude, FSR.latitude)]
gdf = gpd.GeoDataFrame(FSR, crs=4326, geometry=geometry)

In [22]:
FSR.rename(columns={'globalid': 'SRGlobalID', 'closeddate': 'SRClosedDate', 'createddate': 'SRCreatedDate', 'updateddate': 'SRUpdatedDate'}, inplace=True)
FI.rename(columns={'globalid': 'InsGlobalID', 'closeddate': 'InsClosedDate', 'createddate': 'InsCreatedDate', 'updateddate': 'InsUpdatedDate', 'servicerequestglobalid': 'SRGlobalID'}, inplace=True)
FWO.rename(columns={'globalid': 'WOGlobalID', 'closeddate': 'WOClosedDate', 'createddate': 'WOCreatedDate', 'updateddate': 'WOUpdatedDate', 'inspectionglobalid': 'InsGlobalID'}, inplace=True)
FRA.rename(columns={'globalid': 'RAGlobalID', 'closeddate': 'RAClosedDate', 'createddate': 'RACreatedDate', 'inspectionglobalid': 'InsGlobalID'}, inplace=True)

# not further process inspection dataset because only one inspection is needed for under-reporting measuring, but more inspections could still be used for other purposes

# Merge Inspections

In [23]:
FI.columns

Index(['inspectiontype', 'inspectionstatus', 'inspectiontpcondition',
       'inspectiontpstructure', 'treepointdbh', 'InsGlobalID', 'SRGlobalID',
       'inspectiondate', 'InsClosedDate', 'InsCreatedDate', 'InsUpdatedDate',
       'parentinspectionglobalid', 'reinspectiondate', 'location'],
      dtype='object')

In [24]:
FI = FI[FI['location'].notna()]

In [25]:
FI["longitude"] = FI.location.copy().apply(lambda x: float(x['coordinates'][0])).copy()
FI["latitude"] = FI.location.copy().apply(lambda x: float(x['coordinates'][1])).copy()

In [26]:
geometry = [None if math.isnan(xy[0]) or math.isnan(xy[1]) else Point(xy) for xy in zip(FI.longitude, FI.latitude)]
g_FI = gpd.GeoDataFrame(FI, crs=4326, geometry=geometry)

In [27]:
# back to regular df
merge_FI = pd.DataFrame(g_FI)

In [28]:
merge_FI.head()

,inspectiontype,inspectionstatus,inspectiontpcondition,inspectiontpstructure,treepointdbh,InsGlobalID,SRGlobalID,inspectiondate,InsClosedDate,InsCreatedDate,InsUpdatedDate,parentinspectionglobalid,reinspectiondate,location,longitude,latitude,geometry
0,Level 2,Closed,Fair,Full,5,38CA57ED-D91B-4FA9-B00D-CB708391105F,24DA7673-7FE3-403D-A712-A8ED06E69F98,2023-01-02 14:39:26.0000000,2023-01-02 14:39:26.0000000,2023-01-02 14:39:26.0000000,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-73.91293265...",-73.912933,40.611473,POINT (-73.91293 40.61147)
1,Level 2,Closed,Fair,Full,5,E0171F1A-3F1F-4892-ADED-3DDDA7E76333,24DA7673-7FE3-403D-A712-A8ED06E69F98,2023-01-02 14:40:44.0000000,2023-01-02 14:40:44.0000000,2023-01-02 14:40:44.0000000,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-73.91296225...",-73.912962,40.611408,POINT (-73.91296 40.61141)
2,Level 1,Closed,Dead,Shaft,NaN,EB2B4CCE-AC54-41D6-B5EE-78C6004ED55F,NaN,2023-01-02 15:03:20.0000000,2023-01-02 15:03:20.0000000,2023-01-02 15:03:20.0000000,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-73.91404831...",-73.914048,40.605873,POINT (-73.91405 40.60587)
3,Level 1,Closed,Dead,Shaft,NaN,720EA09F-37AE-40EA-B322-E560819FDF4F,NaN,2023-01-02 15:05:05.0000000,2023-01-02 15:05:05.0000000,2023-01-02 15:05:05.0000000,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-73.90972182...",-73.909722,40.604295,POINT (-73.90972 40.60430)
4,Level 1,Closed,Dead,Shaft,NaN,F6F62C86-AFC7-4EB7-B303-305108E99499,NaN,2023-01-02 15:09:42.0000000,2023-01-02 15:09:42.0000000,2023-01-02 15:09:42.0000000,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-73.90891126...",-73.908911,40.610073,POINT (-73.90891 40.61007)


In [29]:
# merge service requests w/ inspections
mergeddf = pd.merge(FSR, merge_FI, on='SRGlobalID', how='left', suffixes=('_SR', '_I'), left_index=False, right_index=False)

In [30]:
#before merging
print("FI size", len(merge_FI))
print("SR size", len(FSR))

FI size 20526
SR size 5542


In [31]:
#after merging
print("merged size", len(mergeddf))

merged size 5680


In [32]:
#Inspections with no SR
a = merge_FI[merge_FI["SRGlobalID"].notna()]
len(a)

##There could be inspections without service requests (no global SRID). There could be many service requests with no inspections.

8420

In [33]:
mergeddf.columns

Index(['srcategory', 'srtype', 'srpriority', 'srsource', 'srstatus',
       'srresolution', 'boroughcode', 'servicerequestparentglobalid',
       'SRGlobalID', 'initiateddate', 'SRClosedDate', 'SRCreatedDate',
       'SRUpdatedDate', 'descriptor1', 'complainttype', 'callerzipcode',
       'srcallertype', 'latitude_SR', 'longitude_SR', 'nta', 'communityboard',
       'zipcode', 'census_tract', 'GLOBAL_ID', 'geometry_SR', 'inspectiontype',
       'inspectionstatus', 'inspectiontpcondition', 'inspectiontpstructure',
       'treepointdbh', 'InsGlobalID', 'inspectiondate', 'InsClosedDate',
       'InsCreatedDate', 'InsUpdatedDate', 'parentinspectionglobalid',
       'reinspectiondate', 'location', 'longitude_I', 'latitude_I',
       'geometry_I'],
      dtype='object')

In [34]:
FSR.shape, merge_FI.shape, mergeddf.shape # since merged > FSR, means there are some SRs with multiple inspections

((5542, 25), (20526, 17), (5680, 41))

In [35]:
mergeddf['InsGlobalID'].fillna('0', inplace=True)

# Merge Work Orders

In [36]:
FWO.columns

Index(['wotype', 'wostatus', 'wopriority', 'boroughcode', 'InsGlobalID',
       'WOGlobalID', 'WOClosedDate', 'canceldate', 'cancelreason',
       'WOCreatedDate', 'WOUpdatedDate', 'woentity', 'woproject', 'wocategory',
       'projstartdate', 'recommendedspecies', 'location'],
      dtype='object')

In [37]:
FWO["longitude"] = FWO.location.copy().apply(lambda x: float(x['coordinates'][0])).copy()
FWO["latitude"] = FWO.location.copy().apply(lambda x: float(x['coordinates'][1])).copy()

geometry = [None if math.isnan(xy[0]) or math.isnan(xy[1]) else Point(xy) for xy in zip(FWO.longitude, FWO.latitude)]
g_FWO = gpd.GeoDataFrame(FWO, crs=4326, geometry=geometry)

In [38]:
# back to regular df
merge_FWO = pd.DataFrame(g_FWO)

In [39]:
# merge SRs+Insps w/ WOs
mergeddf = pd.merge(mergeddf, merge_FWO, on='InsGlobalID', how='left', suffixes=('', '_WO'))
FWO.shape, mergeddf.shape

((20526, 20), (5680, 60))

In [40]:
mergeddf.columns

Index(['srcategory', 'srtype', 'srpriority', 'srsource', 'srstatus',
       'srresolution', 'boroughcode', 'servicerequestparentglobalid',
       'SRGlobalID', 'initiateddate', 'SRClosedDate', 'SRCreatedDate',
       'SRUpdatedDate', 'descriptor1', 'complainttype', 'callerzipcode',
       'srcallertype', 'latitude_SR', 'longitude_SR', 'nta', 'communityboard',
       'zipcode', 'census_tract', 'GLOBAL_ID', 'geometry_SR', 'inspectiontype',
       'inspectionstatus', 'inspectiontpcondition', 'inspectiontpstructure',
       'treepointdbh', 'InsGlobalID', 'inspectiondate', 'InsClosedDate',
       'InsCreatedDate', 'InsUpdatedDate', 'parentinspectionglobalid',
       'reinspectiondate', 'location', 'longitude_I', 'latitude_I',
       'geometry_I', 'wotype', 'wostatus', 'wopriority', 'boroughcode_WO',
       'WOGlobalID', 'WOClosedDate', 'canceldate', 'cancelreason',
       'WOCreatedDate', 'WOUpdatedDate', 'woentity', 'woproject', 'wocategory',
       'projstartdate', 'recommendedspecies', '

# Merge Risk Assessments

In [41]:
# merge SRs + Insps + WOs w/ Risk Assessmts
mergeddf = pd.merge(mergeddf, FRA, on='InsGlobalID', how='left', suffixes=('', '_RA'))
FRA.shape, mergeddf.shape

((14089, 12), (6176, 71))

In [42]:
# drop cols with all null values, if any
# mergeddf = mergeddf.dropna(axis=1, how='all')

In [43]:
mergeddf.columns.values # what we're working with

array(['srcategory', 'srtype', 'srpriority', 'srsource', 'srstatus',
       'srresolution', 'boroughcode', 'servicerequestparentglobalid',
       'SRGlobalID', 'initiateddate', 'SRClosedDate', 'SRCreatedDate',
       'SRUpdatedDate', 'descriptor1', 'complainttype', 'callerzipcode',
       'srcallertype', 'latitude_SR', 'longitude_SR', 'nta',
       'communityboard', 'zipcode', 'census_tract', 'GLOBAL_ID',
       'geometry_SR', 'inspectiontype', 'inspectionstatus',
       'inspectiontpcondition', 'inspectiontpstructure', 'treepointdbh',
       'InsGlobalID', 'inspectiondate', 'InsClosedDate', 'InsCreatedDate',
       'InsUpdatedDate', 'parentinspectionglobalid', 'reinspectiondate',
       'location', 'longitude_I', 'latitude_I', 'geometry_I', 'wotype',
       'wostatus', 'wopriority', 'boroughcode_WO', 'WOGlobalID',
       'WOClosedDate', 'canceldate', 'cancelreason', 'WOCreatedDate',
       'WOUpdatedDate', 'woentity', 'woproject', 'wocategory',
       'projstartdate', 'recommendedspec

# Export to Geojson

In [ ]:
#Sample Format
{
  "features": [
    {
      "type": "Feature",
      "properties": {
        "title": "Lincoln Park",
        "description": "A northside park that is home to the Lincoln Park Zoo"
      },
      "geometry": {
        "coordinates": [-87.637596, 41.940403],
        "type": "Point"
      }
    },
    {
      "type": "Feature",
      "properties": {
        "title": "Burnham Park",
        "description": "A lakefront park on Chicago's south side"
      },
      "geometry": {
        "coordinates": [-87.603735, 41.829985],
        "type": "Point"
      }
    },
  ],
  "type": "FeatureCollection"
}

In [ ]:
# adapted from https://notebook.community/captainsafia/nteract/applications/desktop/example-notebooks/pandas-to-geojson
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'features':[], 'type':'FeatureCollection'}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'coordinates':[],
                               'type': 'Point'}
                  }

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [44]:
mergeddf.columns

Index(['srcategory', 'srtype', 'srpriority', 'srsource', 'srstatus',
       'srresolution', 'boroughcode', 'servicerequestparentglobalid',
       'SRGlobalID', 'initiateddate', 'SRClosedDate', 'SRCreatedDate',
       'SRUpdatedDate', 'descriptor1', 'complainttype', 'callerzipcode',
       'srcallertype', 'latitude_SR', 'longitude_SR', 'nta', 'communityboard',
       'zipcode', 'census_tract', 'GLOBAL_ID', 'geometry_SR', 'inspectiontype',
       'inspectionstatus', 'inspectiontpcondition', 'inspectiontpstructure',
       'treepointdbh', 'InsGlobalID', 'inspectiondate', 'InsClosedDate',
       'InsCreatedDate', 'InsUpdatedDate', 'parentinspectionglobalid',
       'reinspectiondate', 'location', 'longitude_I', 'latitude_I',
       'geometry_I', 'wotype', 'wostatus', 'wopriority', 'boroughcode_WO',
       'WOGlobalID', 'WOClosedDate', 'canceldate', 'cancelreason',
       'WOCreatedDate', 'WOUpdatedDate', 'woentity', 'woproject', 'wocategory',
       'projstartdate', 'recommendedspecies', '

In [45]:
mergeddf["riskrating"]

0       NaN
1       NaN
2       NaN
3       7.0
4       7.0
       ... 
6171    NaN
6172    NaN
6173    NaN
6174    NaN
6175    NaN
Name: riskrating, Length: 6176, dtype: object

In [47]:
cols_to_keep = ['srcategory', 'complainttype', 'srtype', 'descriptor1', 'srpriority', 'srstatus', 'srresolution', 
        'initiateddate', 'SRClosedDate', 'SRCreatedDate', 'SRUpdatedDate', 'SRGlobalID',
        'inspectiontype', 'inspectionstatus', 'inspectiontpcondition', 'inspectiontpstructure', 
        'inspectiondate', 'InsCreatedDate', 'InsUpdatedDate',  'InsClosedDate', 'reinspectiondate', 'parentinspectionglobalid', 'InsGlobalID',
        'WOGlobalID','wotype', 'wostatus', 'wocategory', 'woentity', 'woproject', 'wopriority',
        'WOCreatedDate', 'WOUpdatedDate',  'WOClosedDate', 'cancelreason', 'canceldate', 'projstartdate',
        'radefect', 'radefectlocation', 'failure', 'failureimpact', 'impacttarget', 'consequence', 'riskrating', 
        'RAGlobalID', 'RACreatedDate', 'nta', 'boroughcode', 'communityboard', 'zipcode', 'census_tract', 'latitude_SR', 'longitude_SR'] #, 'geometry_SR']

In [48]:
mergeddf.columns

Index(['srcategory', 'srtype', 'srpriority', 'srsource', 'srstatus',
       'srresolution', 'boroughcode', 'servicerequestparentglobalid',
       'SRGlobalID', 'initiateddate', 'SRClosedDate', 'SRCreatedDate',
       'SRUpdatedDate', 'descriptor1', 'complainttype', 'callerzipcode',
       'srcallertype', 'latitude_SR', 'longitude_SR', 'nta', 'communityboard',
       'zipcode', 'census_tract', 'GLOBAL_ID', 'inspectiontype',
       'inspectionstatus', 'inspectiontpcondition', 'inspectiontpstructure',
       'treepointdbh', 'InsGlobalID', 'inspectiondate', 'InsClosedDate',
       'InsCreatedDate', 'InsUpdatedDate', 'parentinspectionglobalid',
       'reinspectiondate', 'location', 'longitude_I', 'latitude_I', 'geometry',
       'wotype', 'wostatus', 'wopriority', 'boroughcode_WO', 'WOGlobalID',
       'WOClosedDate', 'canceldate', 'cancelreason', 'WOCreatedDate',
       'WOUpdatedDate', 'woentity', 'woproject', 'wocategory', 'projstartdate',
       'recommendedspecies', 'location_WO', 'lo

In [49]:
mergeddf = mergeddf[cols_to_keep]

In [ ]:
#drop rows with missing values
# df = mergeddf.dropna()

# Data Aggregation

## Columns Inspection

In [50]:
#data are based on those with geographic data
#sr category
mergeddf.srcategory.unique()

array(['Plant Tree', 'Hazard', 'Prune', 'Root/Sewer/Sidewalk',
       'Illegal Tree Damage', 'Remove Tree', 'Remove Debris',
       'Remove Stump', 'Rescue/Preservation'], dtype=object)

In [51]:
#complait type
mergeddf.complainttype.unique()

array(['New Tree Request', 'Damaged Tree', 'Overgrown Tree/Branches',
       'Root/Sewer/Sidewalk Condition', 'Illegal Tree Damage', nan,
       'Dead/Dying Tree', 'Wood Pile Remaining', 'Uprooted Stump'],
      dtype=object)

In [52]:
#sr type
mergeddf.srtype.unique()

array(['Street Tree', 'Tree Leaning', 'Limb Down - Other',
       'Prune Dead Wood', 'Clear Building', 'Trees and Sidewalks',
       'Trunk Damaged Bicycle Chained', 'Tree Down - Street',
       'Clear Traffic - Signal', 'Root Damage', 'Tree Down - Other',
       'Trunk Damaged', 'Branches Damaged', 'Blocked Sewer',
       'Limb Down - Sidewalk', 'Clear Road',
       'Hanging Limb from other Branches', 'Hanging Limb',
       'Dead Tree Older Than 2 Years', 'Limb Down - House',
       'Wood Pile Remaining', 'Hanging Limb Other', 'Prune Other',
       'Poor Condition Cracked', 'Limb Down - Car',
       'Clear Utilities - Power Lines', 'Tree Down - Sidewalk',
       'Tree Split', 'Tree Down - Park Passive Area',
       'Tree Down - Park Active Area', 'Tree Uprooted',
       'Cracked Foundation', 'Hanging Limb from Wires',
       'Clear Utilities - Other', 'Poor Condition Hollow',
       'Limb Down - Street', 'Unauthorized Tree Removal',
       'Uprooted Stump', 'Limb Down - Driveway',
   

In [53]:
mergeddf.srresolution.unique()

array(['Reviewed - Inspection Assigned',
       'Inspected - Tentatively Approved for Planting',
       'Inspected - Condition Not Found', 'No Action - Jurisdiction',
       'Inspected - No Work Order', 'Work Priority Category - D', nan,
       'Work Priority Category - C', 'No Action - Insufficient Info',
       'Work Completed', 'Planting Declined - Unplantable',
       'T/S Inspected - No Repair', 'Refer to NYCHA',
       'Reviewed - Block Pruning', 'Refer to MTA',
       'Inspected - Utility Wk Reqd', 'T/S Inspected - Repair Pending',
       'T/S Inspected - Ineligible', 'Work Priority Category - B',
       'Inspected - Location Not Found', 'Planting Declined - Temp',
       'Refer to DEP', 'Storm Debris Removal Pending',
       'Work Priority Category - A', 'Refer to DOT',
       'T/S No Action - Ineligible', 'T/S Inspected - Site Waitlisted',
       'Refer to DOHMH', 'Conflicting Jurisdiction', 'Work Order Created',
       'Work Order Created - Work Priority Category B', 'Refer t

In [54]:
mergeddf.InsCreatedDate.unique()

array([nan, '2023-01-04 18:43:46.0000000', '2023-01-17 18:38:05.0000000',
       ..., '2023-02-06 16:50:31.0000000', '2023-02-02 17:48:12.0000000',
       '2023-02-06 17:24:01.0000000'], dtype=object)

In [55]:
mergeddf.inspectiontype.unique()

array([nan, 'Level 2', 'Level 1', 'Sidewalk Rating',
       'General - Planting Space'], dtype=object)

In [56]:
len(mergeddf)

5726

### Data Insights

In [57]:
len(mergeddf[(mergeddf.InsCreatedDate != "N/A") & (mergeddf.inspectiontype == "N/A")])

0

In [46]:
mergeddf.columns

Index(['srcategory', 'complainttype', 'srtype', 'descriptor1', 'srpriority',
       'srstatus', 'srresolution', 'initiateddate', 'SRClosedDate',
       'SRCreatedDate', 'SRUpdatedDate', 'inspectiontype', 'inspectionstatus',
       'inspectiontpcondition', 'inspectiontpstructure', 'inspectiondate',
       'InsCreatedDate', 'InsUpdatedDate', 'InsClosedDate', 'reinspectiondate',
       'parentinspectionglobalid', 'wotype', 'wostatus', 'wocategory',
       'woentity', 'woproject', 'wopriority', 'WOCreatedDate', 'WOUpdatedDate',
       'WOClosedDate', 'cancelreason', 'canceldate', 'projstartdate',
       'radefect', 'radefectlocation', 'failure', 'failureimpact',
       'impacttarget', 'consequence', 'riskrating', 'RACreatedDate', 'nta',
       'boroughcode', 'communityboard', 'zipcode', 'census_tract',
       'latitude_SR', 'longitude_SR', 'geometry_SR'],
      dtype='object')

## Add columns for graph & aggregation

In [64]:
mergeddf.InsGlobalID.unique()

array(['0', '9196F0A4-2CCE-4C23-A8C7-9510EAFDFED5',
       '17E15316-3726-44F3-8ED2-22E4A766FFC5',
       'A81039C3-27FF-4574-9C44-2CDA06DA9BB1',
       '521D0F53-06ED-434C-A334-E3631FE3FCF8',
       'DDE23D8B-5519-4A36-97D5-D6E151680A59',
       '22132514-D74A-4526-8A9C-73B7FDD1A222',
       'E52A2463-63CA-4BE6-8BCB-CF8D517696C5',
       '567BDF1F-0754-4819-ADA7-0493CB89F5BF',
       '60616DF9-FA9A-4DD5-9CE5-A6F83CDB4DE3',
       'AAE4243B-CD6A-4047-8CB4-25C7AE8B7F48',
       '5DDE3E27-DB43-4148-8903-8B02289F1E35',
       '62117993-218B-46C6-A99E-988E5740A676',
       '4DB5E35C-6927-40CE-BE50-F1F9B71F6B8A',
       '2D1066B7-2106-466E-B03F-756F9E1BFE31',
       '7EBCAA47-FA4A-4239-B422-3CFBBF37BDAE',
       'C75DDDC7-9D18-482B-9535-D0512AC21019',
       'DCCE4FAA-5EF0-4361-87AD-5683E0C3AE52',
       'AA499FC4-704F-499A-8C9F-B410D895A873',
       '5E45C2D5-DF17-405B-A0E4-14D0BEB9D211',
       'E5916E40-FFD2-4615-87B2-C8AB62B29BD1',
       '51C3FB22-02AB-41CC-BF48-91FF913C5769',
       '

In [58]:
mergeddf['inspection_attached'] = np.where(mergeddf['InsGlobalID']!= '0', True, False)

/var/folders/sw/bm9l8xdn4yxdqvn80kxmq7y00000gn/T/ipykernel_19032/611471582.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mergeddf['inspection_attached'] = np.where(mergeddf['InsGlobalID']!= '0', True, False)


In [59]:
mergeddf['wo_attached'] = np.where(mergeddf['WOGlobalID'] != 'N/A', True, False)

/var/folders/sw/bm9l8xdn4yxdqvn80kxmq7y00000gn/T/ipykernel_19032/310566942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mergeddf['wo_attached'] = np.where(mergeddf['WOGlobalID'] != 'N/A', True, False)


In [60]:
mergeddf['initiateddate'] = pd.to_datetime(mergeddf['initiateddate'],errors='coerce')
mergeddf['initiated_year'] = pd.DatetimeIndex(mergeddf['initiateddate']).year
mergeddf['initiated_month'] = pd.DatetimeIndex(mergeddf['initiateddate']).month
mergeddf['SRCreatedDate_datetime'] = pd.to_datetime(mergeddf['SRCreatedDate'],errors='coerce')

/var/folders/sw/bm9l8xdn4yxdqvn80kxmq7y00000gn/T/ipykernel_19032/2699466242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mergeddf['initiateddate'] = pd.to_datetime(mergeddf['initiateddate'],errors='coerce')
/var/folders/sw/bm9l8xdn4yxdqvn80kxmq7y00000gn/T/ipykernel_19032/2699466242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mergeddf['initiated_year'] = pd.DatetimeIndex(mergeddf['initiateddate']).year
/var/folders/sw/bm9l8xdn4yxdqvn80kxmq7y00000gn/T/ipykernel_19032/2699466242.py:3: SettingWit

## Calculate Numbers

### SR Total

In [61]:
#for global
len(mergeddf)

5726

In [62]:
#for each month
count_by_month = mergeddf['SRCreatedDate_datetime'].apply(lambda row: row.month)
count_by_month = count_by_month.to_frame()
count_by_month.rename(columns={'SRCreatedDate_datetime' : 'SRCreated_month'}, inplace = "True")

In [63]:
count_by_month.groupby(["SRCreated_month"]).size()

SRCreated_month
1    4366
2    1360
dtype: int64

### Uninspected Service Requests

In [64]:
uninspected = mergeddf[mergeddf["inspection_attached"] == False]

In [65]:
#For global
len(uninspected)

4396

In [66]:
#For each month
uninspected_by_month = uninspected['SRCreatedDate_datetime'].apply(lambda row: row.month).to_frame()
uninspected_by_month.rename(columns={'SRCreatedDate_datetime' : 'SRCreated_month'}, inplace = "True")


In [67]:
uninspected_by_month.groupby(["SRCreated_month"]).size()

SRCreated_month
1    3175
2    1221
dtype: int64

# Dump data to geojson

In [62]:
#drop rows is there's no geometry info
mergeddf = mergeddf[mergeddf['latitude_SR'].notna()]

In [63]:
mergeddf['complainttype'].fillna('None Selected', inplace=True)
mergeddf['srtype'].fillna('None Selected', inplace=True)
mergeddf['riskrating'].fillna(0, inplace=True)
mergeddf.fillna('N/A', inplace = True)

In [ ]:
geojson = df_to_geojson(mergeddf, cols_to_keep, lat='latitude_SR', lon='longitude_SR')

In [ ]:
fileName = "{}{}".format(last_fetch_time, ".geojson")
with open(fileName, 'w') as outfile:
    dump(geojson, outfile)